# Prototipo de clasificación de reclamos CMF

##### Prototipo para postulación a etapa 1 de "Reto de Innovación de Interés Público SupTech para la Supervisión de Conducta en Mercados Financieros"

El siguiente documento contiene el modelo propuesto para resolver la clasificación binaria de reclamos de valores y seguros, junto con el procesamiento de los 100 reclamos sin etiqueta. El modelo ya fue entrenado con los 1000 reclamos con etiqueta, por lo que en este documento se importan los parámetros ya ajustados. Para mayor información sobre el entrenamiento, favor reisar documento "Entrenamiento de Prototipo".


Autores:
* Bastian Ermann Rodríguez <<bastian@ermannb.com>>
* Pablo Uribe Pizarro <<pablo.uribe@student.ecp.fr>>
* Felipe Uribe Pizarro <<felipe.uribe.pizarro@gmail.com>>

08/02/2021

In [ ]:
# Instalación de packages

pip install -r requirements.txt

## Importación de datos

En primera instancia, se carga archivo excel de 100 reclamos sin clasificación, en formato pandas.

In [1]:
import pandas as pd
import os
cwd = os.getcwd()

file_sclas =pd.read_excel(cwd+r'/articles-40220_recurso_1/reclamos_20201221_sin_clas.xlsx')
file_sclas.set_index('CASO_ID')


# Ejemplo de un reclamo sin procesamiento

print(file_sclas['DESCRIPCION_CIUDADANO'].iloc[0])

Conforme a lo indicado en OFORD N°XX, SGD: N°XX de fecha 12 de noviembre del 2014, envío documentación adjunta, específicamente, certificado en el cual mi jefatura indica que el día 27 de Junio del presente año, el suscrito se encontraba de servicio de guardia, cumpliendo funciones propias de un Funcionario Público de la PDI., Lo anterior conforme al motivo de fuerza mayor enunciado en el Art. 16, número 2 de las condiciones generales de la póliza. Haciendo presente que en el Art N°45 del Código Civil se indica; Se llama fuerza mayor o caso fortuito el imprevisto a que no es posible resistir, como un naufragio, un terremoto, el apresamiento de enemigos, los actos de autoridad ejercidos por un funcionario público, etc. En mi situación por ser funcionario público.
Con respecto a que al momento de la denuncia se indicó que el vehículo siniestrado ya había aparecido, debo indicar que dicha información fue recepcionada telefónicamente durante se realizaba denuncia ante personal de Carabiner

## Preprocesamiento de datos

Tras importar los reclamos, se realiza un preprocesamiento de los mismos previo a ser ingresados al modelo. El objetivo es dejar sólo las palabras más relevantes en el texto a analizar, en un formato estándar. Este paso contempla lo siguiente:

1. Remoción de caracteres fuera del abecedario español (puntuación, números, paréntesis, guiones, etc).
2. Tokenización, es decir, transformar cada reclamo en una lista de palabras (strings).
3. Remoción de "stopwords" (palabras comunes en español, que no agregan mayor información para la clasificación).
4. Estandarización de palabras, transformando todas las letras a minúsculas.

Se imprimen ejemplos de "stopwords" a continuación.

In [2]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('spanish')[:10])

def preprocess(text):

    text = re.sub(r'[^A-Za-zÑñÁáÉéÍíÓóÚú]', ' ', text)
    
    tokens = text.split()
    
    stop_words = stopwords.words('spanish')
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if (token not in stop_words and len(token) > 1)]
    return tokens

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Se aplica el preprocesamiento a las columnas 'DESCRIPCION_CIUDADANO' y 'PETICION_CIUDADANO'

file_sclas['DESCRIPCION_CIUDADANO']=file_sclas['DESCRIPCION_CIUDADANO'].apply(preprocess)
file_sclas['PETICION_CIUDADANO']=file_sclas['PETICION_CIUDADANO'].apply(preprocess)

In [4]:
#Ejemplo de reclamo preprocesado

print(file_sclas['DESCRIPCION_CIUDADANO'].iloc[0])

['conforme', 'indicado', 'oford', 'xx', 'sgd', 'xx', 'fecha', 'noviembre', 'envío', 'documentación', 'adjunta', 'específicamente', 'certificado', 'jefatura', 'indica', 'día', 'junio', 'presente', 'año', 'suscrito', 'encontraba', 'servicio', 'guardia', 'cumpliendo', 'funciones', 'propias', 'funcionario', 'público', 'pdi', 'anterior', 'conforme', 'motivo', 'fuerza', 'mayor', 'enunciado', 'art', 'número', 'condiciones', 'generales', 'póliza', 'haciendo', 'presente', 'art', 'código', 'civil', 'indica', 'llama', 'fuerza', 'mayor', 'caso', 'fortuito', 'imprevisto', 'posible', 'resistir', 'naufragio', 'terremoto', 'apresamiento', 'enemigos', 'actos', 'autoridad', 'ejercidos', 'funcionario', 'público', 'etc', 'situación', 'ser', 'funcionario', 'público', 'respecto', 'momento', 'denuncia', 'indicó', 'vehículo', 'siniestrado', 'aparecido', 'debo', 'indicar', 'dicha', 'información', 'recepcionada', 'telefónicamente', 'realizaba', 'denuncia', 'personal', 'carabineros', 'chile', 'relación', 'denunc

## Vocabulario de entrenamiento

Para poder ingresar los reclamos al modelo, previamente se deben codificar las palabras a IDs, lo que se conoce como un vocabulario. El vocabulario es generado en la fase de entrenamiento del modelo, utilizando las palabras de los 1000 reclamos con etiqueta.

A continuación, se filtran los reclamos sin clasificación con el vocabulario generado en entrenamiento, para luego traducir cada palabra a su respectivo ID.

In [5]:
#Se carga vocabulario

import json
with open('vocab_lstm_v4.json') as json_file: 
    vocab = json.load(json_file)

In [6]:
#Se filtran los reclamos, para que contengan sólo palabras dentro del vocabulario de entrenamiento del modelo

def filter_vocab(text,vocab):

    filtered_text = [word for word in text if (word in vocab)]
    return filtered_text

file_sclas['DESCRIPCION_CIUDADANO']=file_sclas['DESCRIPCION_CIUDADANO'].apply(filter_vocab,vocab = vocab)
file_sclas['PETICION_CIUDADANO']=file_sclas['PETICION_CIUDADANO'].apply(filter_vocab,vocab = vocab)

In [7]:
#Se unen columnas 'DESCRIPCION_CIUDADANO' y 'PETICION_CIUDADANO' en un sólo gran string a ser procesado por el modelo

X_sclas=(file_sclas['DESCRIPCION_CIUDADANO']+file_sclas['PETICION_CIUDADANO']).values

In [8]:
#Se codifican palabras en sus respectivos ID's

X_ints = []
for text in X_sclas:
    X_ints.append([vocab[word] for word in text])

In [9]:
#Ejemplo de texto codificado

print(X_ints[0])

[18, 1465, 3072, 11, 6061, 11, 115, 470, 2187, 278, 351, 3301, 271, 4, 67, 812, 482, 69, 930, 1481, 957, 5857, 2401, 5570, 3154, 2138, 1782, 603, 18, 920, 304, 5093, 77, 20, 21, 10, 1140, 482, 5093, 3138, 2341, 4, 1398, 304, 677, 1514, 2309, 4605, 2783, 2138, 1782, 1757, 580, 451, 2138, 1782, 319, 123, 837, 784, 406, 3175, 5195, 196, 1132, 540, 50, 109, 2789, 2771, 837, 595, 1166, 462, 1983, 2312, 34, 1507, 67, 597, 930, 1507, 512, 4542, 540, 34, 802, 354, 1917, 18, 603, 67, 1054, 2312, 411, 412, 1056, 11, 2298, 897, 421, 67, 249, 67, 2677, 295, 812, 1635, 4, 259, 1275, 489, 1008, 190, 489, 1869, 411, 2, 1219, 67, 909, 1487, 930, 67, 909, 411, 2, 1219, 2251, 355, 1212, 1821, 30, 2183, 406, 3175, 1490, 1056, 1140, 482, 1056, 67, 909, 3797, 2789, 5608, 489, 190, 930, 3888, 406, 3175, 1281, 53, 54, 703, 222, 2020, 115]


## Estandarización de número de palabras en textos

Para que el modelo pueda trabajar con textos que poseen distinto número de palabras, se estandariza cada uno considerando el máximo número de palabras para un reclamo en el entrenamiento (231). Para ello, se rellenan todos los textos con ceros (palabra "vacía") hasta llegar al largo estándar.

In [10]:
def pad_features(text_ints, seq_length):
    
    features = np.zeros((len(text_ints), seq_length), dtype=int)

    for i, row in enumerate(text_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [11]:
import numpy as np
seq_length = 231

features = pad_features(X_ints, seq_length=seq_length)

assert len(features)==len(X_ints), "Número de filas no coinciden."
assert len(features[0])==seq_length, "Número de columnas no coinciden."

# Ejemplo de texto estandarizado

print(features[:1,:])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0   18 1465 3072   11 6061   11  115  470 2187  278  351 3301
   271    4   67  812  482   69  930 1481  957 5857 2401 5570 3154 2138
  1782  603   18  920  304 5093   77   20   21   10 1140  482 5093 3138
  2341    4 1398  304  677 1514 2309 4605 2783 2138 1782 1757  580  451
  2138 1782  319  123  837  784  406 3175 5195  196 1132  540   50  109
  2789 2771  837  595 1166  462 1983 2312   34 1507   67  597  930 1507
   512 4542  540   34  802  354 1917   18  603   67 1054 2312  411  412
  1056   11 2298  897  421   67  249   67 2677  295  812 1635    4  259
  1275  489 1008  190  489 1869  411    2 1219   67  909 1487  9

## Modelo de clasificación

La base del modelo corresponde a un tipo de red neuronal recurrente, llamada LSTM. Esta clase de red neuronal es capaz de procesar información de una secuencia de palabras dentro de un texto. La arquitectura de la red se ilustra a continuación.

<img src='lstm.png' width="600" height="600">

En primer lugar, se utiliza una capa de embedding, que realiza una representación de cada ID del vocabulario en un vector. Esta capa tiene dos finalidades: representar semánticamente la palabra (vectores cercanos poseen contextos similares), y reducir la dimensión de entrada a la red LSTM.

A continuación, los embeddings se pasan a las celdas LSTM. Esta capa genera dos salidas: un output que se pasa a la capa siguiente, que actúa como clasificador propiamente tal; y un segundo valor que es pasado a la misma LSTM, que cumple el rol de ser la "memoria" de largo plazo de la red. Este último punto es el que permite que las redes LSTM tengan buenos resultados para problemas con secuencias de datos.

Finalmente, los outputs de la etapa anterior se pasan a una capa sigmoide. Al tratarse de un problema de clasificación binaria, las clases se representan como 0 (valores) o 1 (seguros). La capa sigmoide entrega un número entre 0 y 1, interpretable como la probabilidad de pertenecer a cada clase. En el esquema se ilustra que se obtiene un único output (al ingresar la útima palabra), ya que se busca que el modelo entrege la clasificación una vez hayan ingresado todas las palabras del reclamo.

In [12]:
import torch.nn as nn

class lstm_classifier(nn.Module):
  
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        
        super(lstm_classifier, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        
        batch_size = x.size(0)

        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)

        sig_out = self.sig(out)
        
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1]
        
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [13]:
import torch

# Instanciar el modelo con sus hiperparámetros

vocab_size = len(vocab)+1 # +1 por la palabra 0 "vacía"
output_size = 1
embedding_dim = 200
hidden_dim = 256
n_layers = 2

net = lstm_classifier(vocab_size, output_size, embedding_dim, hidden_dim, n_layers,drop_prob=0.7)

# Se cargan parámetros internos del modelo, obtenidos a través del entrenamiento

net.load_state_dict(torch.load('lstm_v4.pth'))

# Imprime arquitecura del modelo

print(net)

lstm_classifier(
  (embedding): Embedding(6327, 200)
  (lstm): LSTM(200, 256, num_layers=2, batch_first=True, dropout=0.7)
  (dropout): Dropout(p=0.7, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [14]:
data = torch.from_numpy(features)
h = net.init_hidden(len(X_sclas))
net.eval()

# Se obtiene output de probabilidades de pertenecer a cada clase

output, h = net(data, h)
    
# Se convierten las probabilidades en clases (0 or 1)

pred = torch.round(output.squeeze()) 

# Se muestra output

print(pred)

tensor([1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0.,
        0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1.,
        0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1.,
        0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1.,
        1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 0., 0., 1., 1., 1.], grad_fn=<RoundBackward>)


In [15]:
# Verificación de número de clasificaciones realizadas

print("Número de Clasificaciones: " + str(len(pred)))

# Distrubución de clases en la predicción

print("Distribución: {:.2f}".format(np.mean(pred.detach().numpy())))

Número de Clasificaciones: 100
Distribución: 0.68


In [16]:
#Exportación de datos a csv para ser anexados a archivo Excel

x_np = pred.detach().numpy()
x_df = pd.DataFrame(x_np)
x_df.to_csv('lstm_output.csv')

## Conclusiones

A modo de resumen, la metodología de utilización del prototipo para la clasificación de nuevos reclamos es la siguiente:
1. Importación de datos
2. Preprocesamiento de datos
3. Filtro de nuevos reclamos con el vocabulario de entrenamiento
4. Estandarización de largo de reclamos
5. Intanciamiento de arquitecura del modelo y sus hiperparámetros, e importación de parámetros internos
6. Obtención de la predicción del modelo

La predicción obtenida a partir del modelo muestra los siguientes valores:
* Número de Clasificaciones: 100
* Distribución: 68% de reclamos clasificados como 'APIA -Reclamo Seguros ' (32% como 'Reclamo Valores')

Por lo tanto, el modelo fue capaz de entregar una clasificación a la totalidad de reclamos nuevos. Se observa que la predicción posee una distribución similar a la observada en el conjunto de 1000 datos de entrenamiento (66% para 'APIA -Reclamo Seguros '), lo cual es una indicación de consistencia del modelo. Sin embargo, pueden existir diferencias con las métricas estimadas en el entrenamiento para la presente clasificación, dado que éste fue realizado con un pequeño número de datos. A medida que el modelo sea entrenado con un mayor volumen de datos, se puede obtener una mayor certeza de que las métricas de entrenamiento representen correctamente su performance con nuevos reclamos.